Mukhammadkodir Abdusalomov 474664 and Majidov Elbek 474570

# Blogging App with Streamlit

## Introduction

In today's digital world, blogging has become an essential way to share knowledge, ideas, and experiences. Whether for personal use or business purposes, a blog provides an accessible platform to connect with a wider audience. Building a simple yet efficient blogging app can be a great way to learn about web development, database management, and user interaction.

I chose **Streamlit** for building this blogging app because of its ease of use and rapid development capabilities. Streamlit allows for quick and efficient deployment of Python-based applications with minimal setup. It enables developers to focus more on building the app's core functionality rather than worrying about the complexities of traditional web frameworks.

### Why Streamlit?

- **Fast Development:** Streamlit allows for building interactive apps with just a few lines of Python code. This made it an ideal choice for this project, where I wanted to focus on creating a feature-rich yet simple blogging platform.
  
- **Built-in Interactivity:** Streamlit provides built-in widgets for user interaction, such as forms, buttons, and sliders, that make it easy to create a dynamic app without needing to write complex JavaScript or HTML.
  
- **Integration with Data Science Libraries:** Streamlit is well-suited for projects that involve data analysis, machine learning, and visualization, as it integrates seamlessly with libraries like Pandas, Matplotlib, and Plotly. This flexibility was beneficial when adding features like word clouds or displaying analytics.

- **Easy Deployment:** With minimal configuration, Streamlit apps can be deployed easily on cloud platforms such as Streamlit Cloud or Heroku. This makes sharing your app with others quick and straightforward.

### Features of the Blogging App

This app enables users to:
- Register and log in securely.
- Create, view, and manage blog posts.
- Interact with blogs by adding comments, likes, and tags.
- Track likes and comments, allowing for community engagement.
  
By utilizing **SQLite** as the database backend, I have created a seamless connection between the frontend and the data storage layer, ensuring efficient data retrieval and management.

Let's dive into the features and functionalities of the app!


# Installing Dependencies

Before running the app, you need to install the necessary Python libraries. To make sure everything works smoothly, you should run the following cells to install the required dependencies.

In [18]:
!pip install streamlit

In [ ]:
!pip install jupytext

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Approach to Run Streamlit from Jupyter Notebook

Streamlit apps are traditionally run via a Python script with the `.py` extension. However, Jupyter Notebooks (`.ipynb`) are not directly compatible with Streamlit's execution. To overcome this issue, I developed a custom approach that allows me to run the Streamlit app directly from a Jupyter Notebook.
### Convert Jupyter Notebook to Python Script
Since Jupyter doesn't natively support running Streamlit apps, I used **Jupytext**, a tool that allows converting `.ipynb` files into `.py` files. This conversion allows us to run the Streamlit app from the notebook by executing the Python script.

To convert the notebook into a Python script, I use the following command:

```bash
jupytext --to py your_notebook.ipynb

In [7]:
try:
    import streamlit as st
    import sqlite3
    import pandas as pd
    import hashlib
    import matplotlib.pyplot as plt
    from wordcloud import WordCloud
    from datetime import datetime
    import os
    
    # Database Connection
    conn = sqlite3.connect('data.db')
    cur = conn.cursor()
    
    # Utility Functions
    def hash_password(password):
        return hashlib.sha256(password.encode()).hexdigest()
    
    def verify_password(password, hashed):
        return hashed == hashlib.sha256(password.encode()).hexdigest()
    
    # Table Creation
    def create_tables():
        cur.execute('''
            CREATE TABLE IF NOT EXISTS users(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                username TEXT UNIQUE,
                password TEXT,
                email TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS blogs(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                author_id INTEGER,
                title TEXT,
                article TEXT,
                post_date DATE,
                FOREIGN KEY (author_id) REFERENCES users(id)
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS comments(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                blog_id INTEGER,
                user_id INTEGER,
                comment TEXT,
                post_date DATE,
                FOREIGN KEY (blog_id) REFERENCES blogs(id),
                FOREIGN KEY (user_id) REFERENCES users(id)
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS likes(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                blog_id INTEGER,
                user_id INTEGER,
                FOREIGN KEY (blog_id) REFERENCES blogs(id),
                FOREIGN KEY (user_id) REFERENCES users(id)
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS tags(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                blog_id INTEGER,
                tag TEXT,
                FOREIGN KEY (blog_id) REFERENCES blogs(id)
            )
        ''')
        conn.commit()
    
    # Authentication Functions
    
    
    def register_user(username, password, email):
        hashed_password = hash_password(password)
        cur.execute('INSERT INTO users(username, password, email) VALUES (?, ?, ?)',
                    (username, hashed_password, email))
        conn.commit()
    
    
    def login_user(username, password):
        cur.execute('SELECT password FROM users WHERE username = ?', (username,))
        data = cur.fetchone()
        if data and verify_password(password, data[0]):
            return True
        return False
    
    
    def get_user_id(username):
        cur.execute('SELECT id FROM users WHERE username = ?', (username,))
        data = cur.fetchone()
        return data[0] if data else None
    
    # Blog Functions
    
    
    def add_blog(author_id, title, article, post_date):
        cur.execute('INSERT INTO blogs(author_id, title, article, post_date) VALUES (?, ?, ?, ?)',
                    (author_id, title, article, post_date))
        conn.commit()
    
    
    def view_all_blogs():
        cur.execute('''
            SELECT blogs.id, users.username, blogs.title, blogs.article, blogs.post_date
            FROM blogs
            JOIN users ON blogs.author_id = users.id
        ''')
        return cur.fetchall()
    
    
    def delete_blog(blog_id):
        # Delete related comments and likes first
        cur.execute('DELETE FROM comments WHERE blog_id = ?', (blog_id,))
        cur.execute('DELETE FROM likes WHERE blog_id = ?', (blog_id,))
        cur.execute('DELETE FROM tags WHERE blog_id = ?', (blog_id,))
        # Delete the blog itself
        cur.execute('DELETE FROM blogs WHERE id = ?', (blog_id,))
        conn.commit()
    
    
    def add_comment(blog_id, user_id, comment, post_date):
        cur.execute('INSERT INTO comments(blog_id, user_id, comment, post_date) VALUES (?, ?, ?, ?)',
                    (blog_id, user_id, comment, post_date))
        conn.commit()
    
    
    def get_comments(blog_id):
        cur.execute('''
            SELECT comments.comment, users.username, comments.post_date
            FROM comments
            JOIN users ON comments.user_id = users.id
            WHERE comments.blog_id = ?
        ''', (blog_id,))
        return cur.fetchall()
    
    
    def add_like(blog_id, user_id):
        cur.execute('INSERT INTO likes(blog_id, user_id) VALUES (?, ?)',
                    (blog_id, user_id))
        conn.commit()
    
    
    def count_likes(blog_id):
        cur.execute('SELECT COUNT(*) FROM likes WHERE blog_id = ?', (blog_id,))
        return cur.fetchone()[0]
    
    # Streamlit App
    
    
    def main():
        create_tables()
        st.set_page_config(page_title="Blogging App", layout="wide")
        st.title("Blogging App with Enhanced Features")
    
        # Sidebar Navigation
        menu = ["Home", "View Blogs", "Add Blog",
                "Manage Blogs", "Comments & Likes"]
        auth_menu = ["Login", "Register", "Logout"]
    
        if "logged_in" not in st.session_state:
            st.session_state.logged_in = False
            st.session_state.username = None
            st.session_state.user_id = None
    
        auth_choice = st.sidebar.radio("Authentication", auth_menu)
    
        if auth_choice == "Login":
            st.sidebar.subheader("Login")
            username = st.sidebar.text_input("Username")
            password = st.sidebar.text_input("Password", type="password")
            if st.sidebar.button("Login"):
                if login_user(username, password):
                    st.success(f"Welcome {username}!")
                    st.session_state.logged_in = True
                    st.session_state.username = username
                    st.session_state.user_id = get_user_id(username)
                else:
                    st.error("Invalid username or password")
    
        elif auth_choice == "Register":
            st.sidebar.subheader("Register")
            username = st.sidebar.text_input("Username")
            email = st.sidebar.text_input("Email")
            password = st.sidebar.text_input("Password", type="password")
            if st.sidebar.button("Register"):
                try:
                    register_user(username, password, email)
                    st.success("Registration Successful!")
                except sqlite3.IntegrityError:
                    st.error("Username already exists. Please choose a different one.")
    
        elif auth_choice == "Logout":
            st.session_state.logged_in = False
            st.session_state.username = None
            st.session_state.user_id = None
            st.sidebar.success("Logged out successfully")
    
        # Main App
        if st.session_state.logged_in:
            choice = st.sidebar.radio("Main Menu", menu)
    
            if choice == "Home":
                image_path = os.path.join(os.getcwd(), "post.jpg")
                if os.path.exists(image_path):
                    st.image(image_path, use_container_width=False)
                else:
                    st.warning(
                        "Welcome to the Blogging App! The image is missing.")
                    st.file_uploader("Upload a welcome image (optional):")
    
            elif choice == "View Blogs":
                st.subheader("All Blogs")
                blogs = view_all_blogs()
                for blog in blogs:
                    st.write(f"**Title:** {blog[2]}")
                    st.write(f"**Author:** {blog[1]}")
                    st.write(f"**Date:** {blog[4]}")
                    st.write(f"**Content:** {blog[3]}")
    
                    comments = get_comments(blog[0])
                    st.write("**Comments:**")
                    for comment in comments:
                        st.write(f"- {comment[1]}: {comment[0]} ({comment[2]})")
    
                    likes = count_likes(blog[0])
                    st.write(f"**Likes:** {likes}")
    
                    # Add comment section
                    if st.text_input(f"Add a comment to blog {blog[2]}"):
                        user_comment = st.text_area("Your comment")
                        if st.button("Post Comment", key=f"comment_{blog[0]}"):
                            add_comment(blog[0], st.session_state.user_id,
                                        user_comment, datetime.now().date())
                            st.success("Comment added successfully!")
    
                    # Add like functionality
                    if st.button("Like", key=f"like_{blog[0]}"):
                        add_like(blog[0], st.session_state.user_id)
                        st.success("Blog liked successfully!")
    
                    st.markdown("---")
    
            elif choice == "Add Blog":
                st.subheader("Add a New Blog")
                title = st.text_input("Blog Title")
                article = st.text_area("Blog Content", height=300)
                post_date = st.date_input("Post Date", value=datetime.now().date())
                if st.button("Submit"):
                    add_blog(st.session_state.user_id, title, article, post_date)
                    st.success("Blog added successfully!")
    
            elif choice == "Manage Blogs":
                st.subheader("Manage Blogs")
                blogs = view_all_blogs()
                blog_titles = {blog[2]: blog[0]
                               for blog in blogs if blog[1] == st.session_state.username}
                selected_blog = st.selectbox(
                    "Select Blog to Delete", list(blog_titles.keys()))
                if st.button("Delete"):
                    delete_blog(blog_titles[selected_blog])
                    st.success("Blog deleted successfully!")
    
            elif choice == "Comments & Likes":
                st.subheader("Manage Comments & Likes")
                blogs = view_all_blogs()
                for blog in blogs:
                    if blog[1] == st.session_state.username:
                        st.write(f"**Title:** {blog[2]}")
                        comments = get_comments(blog[0])
                        st.write("**Comments:**")
                        for comment in comments:
                            st.write(
                                f"- {comment[1]}: {comment[0]} ({comment[2]})")
                        likes = count_likes(blog[0])
                        st.write(f"**Total Likes:** {likes}")
                        st.markdown("---")
        else:
            st.warning("Please log in to access the app.")
    
    
    if __name__ == "__main__":
        main()
except Exception as error:
    pass

In [9]:
!jupytext --to py final_project.ipynb

[jupytext] Reading final_project.ipynb in format ipynb
[jupytext] Writing final_project.py (destination file replaced)


# Running the Streamlit App

Now that we have successfully converted the Jupyter notebook to a Python script, we can run the Streamlit app directly from this notebook
it

To execute the converted Python script and launch the Streamlit app, simply run the following cell. This will initiate the Streamlit server and open the app in a new tab or browser window.

```bash
!streamlit run your_noebook.py


In [11]:
!streamlit run final_project.py

^C


# Teamwork and Collaboration

This project was a collaborative effort between **Mukhammadkodir Abdusalomov** and **Elbek Majidov**. While it may seem simple on the surface, we put in a lot of hard work and thought into the development of this blogging app. We carefully planned the ideologies, how each function should operate, the database schema, and much more. Our goal was to create a seamless and efficient blogging platform, and every aspect of the app was shaped by our combined efforts.

### Roles and Responsibilities

- **Mukhammadkodir Abdusalomov** was primarily responsible for the **Python part** of the app. He worked on developing the core functionality, including the app’s logic, authentication system, blog management, and integration with the SQLite database.
  
- **Elbek Majidov** focused on the **user management** aspect of the app and was responsible for **SQL-related tasks**. This included designing the database schema, creating the necessary tables, and managing the relationships between users, blogs, comments, likes, and tags.

### Effective Collaboration

We made sure to communicate effectively and regularly throughout the project, discussing every detail to ensure the app functions as intended. Our collaborative approach included:

- **Git and GitHub**: We used Git and GitHub for version control and to ensure smooth collaboration. This allowed us to work on different parts of the project simultaneously while keeping track of changes, merging work, and resolving any conflicts efficiently.

- **Streamlit Documentation**: To better understand Streamlit’s capabilities, we referred to the official Streamlit documentation. This helped us learn about its powerful features, widgets, and customization options, enabling us to make the most of the framework for our app.

- **Constant Feedback**: We frequently reviewed each other’s work, provided feedback, and adjusted the design and functionality to improve the app.

### Conclusion

Building this app was a great learning experience for both of us, and it wouldn’t have been possible without our teamwork and dedication. The combination of Mukhammadkodir's Python expertise and Elbek’s database management skills allowed us to create a functional and interactive blogging platform. Working together, we were able to overcome challenges and bring this project to life.
